In [2]:
from itertools import islice
import pandas as pd
from sentence_transformers import SentenceTransformer
from typing import Dict

from nomic.atlas import AtlasDataset
from latentsae import Sae


/opt/miniconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Triton not installed, using eager implementation of SAE decoder.


In [3]:
sae_model = Sae.load_from_hub("enjalot/sae-nomic-text-v1.5-FineWeb-edu-100BT", "64_32")

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Dropping extra args {'signed': False}


In [4]:
emb_model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

<All keys matched successfully>


In [5]:
device = "mps"
sae_model = sae_model.to(device)
emb_model = emb_model.to(device)

# Test the SAE

In [6]:
loaded_features = pd.read_parquet("features.parquet").to_dict(orient='records')

In [12]:
def aggregate_encoder_output(encoder_output, k: int = 5) -> Dict[int, float]:
    total_activations = {}
    for idx, act in zip(encoder_output.top_indices.cpu().flatten(), encoder_output.top_acts.cpu().flatten()):
        idx_int = idx.item()
        if idx_int in total_activations:
            total_activations[idx_int] += act.item()
        else:
            total_activations[idx_int] = act.item()
    sorted_activations = dict(sorted(total_activations.items(), key=lambda item: item[1], reverse=True))
    return sorted_activations

In [17]:
def summarize_encoder_output(sorted_activations, k=5):
    return [loaded_features[idx]['label'] for idx in list(islice(sorted_activations, k))]

In [18]:
test_strings = ['baseball', 'football', 'basketball', 'hockey', 'soccer']

In [19]:
summarize_encoder_output(aggregate_encoder_output(sae_model.encode(emb_model.encode(test_strings, convert_to_tensor=True))))

['sports activism and social impact',
 'Cultural narratives and storytelling techniques',
 'interdisciplinary teamwork and project management strategies',
 'quantum computing and nanomaterial advancements',
 'climate classification and temperature analysis']

# Notes

gpt4o-mini overuses the words "interdisciplinary" and "quantum"

Manual nomencodes

5507: Apple, Inc.